# 기본환경 설정

In [ ]:
# !pip install faiss-cpu

In [ ]:
from google.colab import userdata
HF_KEY = userdata.get("HF_KEY")

In [ ]:
import huggingface_hub
huggingface_hub.login(HF_KEY)

# 모델 로딩

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo langchain-community pypdf langchain_huggingface faiss-cpu
!pip install --no-deps unsloth

In [ ]:
from unsloth import FastModel
from langchain.embeddings import HuggingFaceEmbeddings
import torch

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 1024*5, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    # device_map = {"": device}
)

In [ ]:
model = FastModel.for_inference(model)

# Custom ChatModel 함수

In [ ]:
from typing import Any, Dict, Iterable, Iterator, List, Literal, Optional, Type, Union
from pydantic import BaseModel as PydanticBaseModel
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import AIMessage, AIMessageChunk, HumanMessage, SystemMessage, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatResult, ChatGenerationChunk
from langchain_core.runnables import RunnableLambda
from transformers import TextIteratorStreamer
import threading
import re

In [ ]:
class GemmaChatModel(BaseChatModel):
    def __init__(self, model, tokenizer, max_tokens: int = 512, do_sample: bool = True, temperature: float = 0.7, top_p: float = 0.9, verbose: bool = False, **kwargs: Any):
        super().__init__()
        object.__setattr__(self, "model", model)
        object.__setattr__(self, "tokenizer", tokenizer)
        object.__setattr__(self, "max_tokens", max_tokens)
        object.__setattr__(self, "do_sample", do_sample)
        object.__setattr__(self, "temperature", temperature)
        object.__setattr__(self, "top_p", top_p)
        object.__setattr__(self, "verbose", verbose)
        object.__setattr__(self, "_gen_lock", threading.Lock())

    ### 공통 유틸 ###########################################################################
    @property
    def _llm_type(self) -> str:
        return "gemma-chat"

    def _messages_to_conv(self, messages: List[BaseMessage]) -> List[Dict[str, str]]:
        conv: List[Dict[str, str]] = []
        for m in messages:
            if isinstance(m, SystemMessage):
                conv.append({"role": "system", "content": m.content})
            elif isinstance(m, HumanMessage):
                conv.append({"role": "user", "content": m.content})
            elif isinstance(m, AIMessage):
                conv.append({"role": "assistant", "content": m.content})
        return conv

    def _format_messages(self, messages: List[BaseMessage]) -> str:
        if hasattr(self.tokenizer, "apply_chat_template"):
            conv = self._messages_to_conv(messages)
            formatted = self.tokenizer.apply_chat_template(
                conv,
                tokenize=False,
                add_generation_prompt=True,  # 어시스턴트 턴 시작만 넣고 종료 토큰은 모델이 생성
            )
            return formatted

        # 폴백(토크나이저가 템플릿을 제공하지 않을 때만 사용)
        prompt = ""
        for m in messages:
            if isinstance(m, SystemMessage):
                prompt += "<start_of_turn>system\n" + m.content + "<end_of_turn>\n"
            elif isinstance(m, HumanMessage):
                prompt += "<start_of_turn>user\n" + m.content + "<end_of_turn>\n"
            elif isinstance(m, AIMessage):
                prompt += "<start_of_turn>assistant\n" + m.content + "<end_of_turn>\n"
        prompt += "<start_of_turn>assistant\n"
        return prompt

    def _apply_stop(self, text: str, stop: Optional[List[str]]) -> str:
        if not stop:
            return text
        cut = len(text)
        for s in stop:
            idx = text.find(s)
            if idx != -1:
                cut = min(cut, idx)
        return text[:cut]

    def _build_gen_kwargs(self, **kwargs: Any) -> Dict[str, Any]:
        pad_id = self.tokenizer.pad_token_id
        if pad_id is None:
            pad_id = self.tokenizer.eos_token_id

        eot_id = None
        try:
            eot_id = self.tokenizer.convert_tokens_to_ids("<end_of_turn>")
            if isinstance(eot_id, list):
                eot_id = None
        except Exception:
            eot_id = None

        return {
            "max_new_tokens": kwargs.get("max_tokens", self.max_tokens),
            "do_sample": kwargs.get("do_sample", self.do_sample),
            "temperature": kwargs.get("temperature", self.temperature),
            "top_p": kwargs.get("top_p", self.top_p),
            "eos_token_id": eot_id or self.tokenizer.eos_token_id,
            "pad_token_id": pad_id,
        }

    ### Invoke ############################################################################
    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> ChatResult:
        prompt = self._format_messages(messages)
        if getattr(self, "verbose", False):
            print("\n[GemmaChatModel/_generate] ==== FINAL PROMPT ====")
            print(prompt)
            print("=================================================\n")

        # [CHANGED] 기본 stop 시퀀스에 종료 마커 추가
        if stop is None:
            stop = ["</s>", "<end_of_turn>", "<|endoftext|>"]

        with self._gen_lock:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            gen_kwargs = self._build_gen_kwargs(**kwargs)
            with torch.no_grad():
                outputs = self.model.generate(**inputs, **gen_kwargs)

        # [CHANGED] 입력 길이 이후 생성 토큰만 디코딩
        in_len = inputs["input_ids"].shape[1]
        gen_tokens = outputs[0][in_len:]
        decoded = self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

        # [CHANGED] 후단 종료 마커 정리(안전망)
        decoded = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", decoded)

        decoded = self._apply_stop(decoded, stop)
        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=decoded))])

    ### Batch #############################################################################
    def _generate_batch(self, messages_list: List[List[BaseMessage]], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> List[ChatResult]:
        """
        여러 개의 대화를 한 번에 패딩 인코딩하여 generate 가속.
        """
        # [CHANGED] 각 대화를 chat template로 포맷
        prompts = [self._format_messages(msgs) for msgs in messages_list]

        # padding=True, truncation=True 로 배치 인코딩
        tokenized = self.tokenizer(
            prompts, return_tensors="pt", padding=True, truncation=True
        )
        tokenized = {k: v.to(self.model.device) for k, v in tokenized.items()}
        attn = tokenized.get("attention_mask", None)

        gen_kwargs = self._build_gen_kwargs(**kwargs)

        # [CHANGED] 기본 stop 시퀀스
        if stop is None:
            stop = ["</s>", "<end_of_turn>", "<|endoftext|>"]

        with torch.no_grad():
            outputs = self.model.generate(**tokenized, **gen_kwargs)

        results: List[ChatResult] = []
        for i in range(len(prompts)):
            # 각 샘플의 프롬프트 길이만큼 잘라서 신규 토큰만 디코딩
            if attn is not None:
                in_len = int(attn[i].sum().item())
            else:
                in_len = tokenized["input_ids"][i].shape[0]

            gen_tokens = outputs[i][in_len:]
            text = self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
            text = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", text)  # [CHANGED]
            text = self._apply_stop(text, stop)

            results.append(
                ChatResult(generations=[ChatGeneration(message=AIMessage(content=text))])
            )
        return results

    ### Stream ############################################################################
    def _stream(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> Iterator[ChatGenerationChunk]:
        """
        LangChain의 Runnable .stream() 에서 호출되는 내부 스트리밍 제너레이터.
        각 토큰 델타를 ChatGenerationChunk(AIMessageChunk) 로 내보냅니다.
        """
        prompt = self._format_messages(messages)
        if getattr(self, "verbose", False):
            print("\n[GemmaChatModel/_stream] ==== FINAL PROMPT ====")
            print(prompt)
            print("================================================\n")

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        gen_kwargs = self._build_gen_kwargs(**kwargs)

        # transformers 스트리머 설정
        streamer = TextIteratorStreamer(
            self.tokenizer, skip_prompt=True, skip_special_tokens=True
        )

        # [CHANGED] 기본 stop 시퀀스
        if stop is None:
            stop = ["</s>", "<end_of_turn>", "<|endoftext|>"]

        # generate를 백그라운드에서 수행
        def _worker():
            with torch.no_grad():
                self.model.generate(**inputs, **gen_kwargs, streamer=streamer)

        th = threading.Thread(target=_worker, daemon=True)
        th.start()

        # 누적 후 stop 시퀀스까지 안전하게 잘라서 내보내기
        buffer = ""
        emitted = 0

        for piece in streamer:
            buffer += piece
            # [CHANGED] 실시간으로 후단 종료 마커 제거(시각적 잔여물 방지)
            tmp = re.sub(r"(?:</s>|<\|endoftext\|>|<end_of_turn>)+\s*$", "", buffer)
            trimmed = self._apply_stop(tmp, stop)

            # 새로 생긴 구간만 델타로 방출
            if len(trimmed) > emitted:
                delta = trimmed[emitted:]
                emitted = len(trimmed)
                yield ChatGenerationChunk(message=AIMessageChunk(content=delta))

            # stop 시퀀스 감지되면 중단
            if stop and len(trimmed) < len(buffer):
                break

        # 스레드 정리(최대한 조용히 종료 대기)
        th.join(timeout=0.1)

    ### Structured output #################################################################
    def _build_json_system_prompt(self, schema_text: str) -> str:
        # 모델이 JSON만 내도록 강하게 지시 (hallucination 방지용 규칙 포함)
        return (
            "You are a strict JSON generator.\n"
            "Return ONLY a single JSON object, no prose, no backticks, no explanations.\n"
            "Do not include trailing commas. Do not include comments.\n"
            "Conform exactly to the following JSON schema (fields, types, required):\n"
            f"{schema_text}\n"
        )

    def _ensure_pydantic(self):
        if PydanticBaseModel is None:
            raise RuntimeError(
                "Pydantic is not available. Install pydantic or pass a dict schema instead of a BaseModel."
            )

    def _schema_to_text(self, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]]) -> str:
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(
            schema, PydanticBaseModel
        ):
            try:
                json_schema = schema.model_json_schema()  # pydantic v2
            except Exception:
                json_schema = schema.schema()  # pydantic v1
            return json.dumps(json_schema, ensure_ascii=False, indent=2)
        elif isinstance(schema, dict):
            return json.dumps(schema, ensure_ascii=False, indent=2)
        else:
            raise TypeError(
                "schema must be a Pydantic BaseModel subclass or a dict JSON schema."
            )

    def _parse_structured(self, text: str, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]], include_raw: bool):
        # 코드블록 등 제거 시도(혹시 들어올 경우)
        t = text.strip()
        if t.startswith("```"):
            # ```json ... ``` 또는 ``` ... ```
            t = t.strip("`")
            # 첫 줄에 json 명시가 들어있을 수 있음
            t = "\n".join(
                line for line in t.splitlines() if not line.lower().startswith("json")
            )
        # JSON 파싱
        obj = json.loads(t)

        # Pydantic 검증
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(
            schema, PydanticBaseModel
        ):
            validated = (
                schema.model_validate(obj)
                if hasattr(schema, "model_validate")
                else schema.parse_obj(obj)
            )
            return {"parsed": validated, "raw": text} if include_raw else validated
        else:
            # dict 스키마는 별도 검증 없이 반환 (원하면 jsonschema로 검증 가능)
            return {"parsed": obj, "raw": text} if include_raw else obj

    def with_structured_output(self, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]], *, method: Literal["json_mode"] = "json_mode", include_raw: bool = False, system_prefix: Optional[str] = None, deterministic: bool = True):
        schema_text = self._schema_to_text(schema)
        sys_prompt = self._build_json_system_prompt(schema_text)
        if system_prefix:
            sys_prompt = system_prefix.rstrip() + "\n\n" + sys_prompt

        def _invoke(messages_or_any):
            # 입력을 메시지 리스트로 정규화
            if isinstance(messages_or_any, list) and all(
                isinstance(m, BaseMessage) for m in messages_or_any
            ):
                msgs = [SystemMessage(content=sys_prompt)] + messages_or_any
            else:
                # 문자열/딕셔너리 등도 처리
                msgs = [
                    SystemMessage(content=sys_prompt),
                    HumanMessage(content=str(messages_or_any)),
                ]

            # 결정론 옵션
            kw = {}
            if deterministic:
                kw = {"do_sample": False, "temperature": 0.0, "top_p": 1.0}

            # 1차 시도
            result = self._generate(msgs, **kw)
            text = result.generations[0].message.content
            try:
                return self._parse_structured(text, schema, include_raw)
            except Exception:
                # 재시도: 더 강한 지시
                retry_msgs = [
                    SystemMessage(
                        content=sys_prompt + "\nOutput must be valid JSON. Try again."
                    )
                ] + msgs[1:]
                result2 = self._generate(retry_msgs, **kw)
                text2 = result2.generations[0].message.content
                return self._parse_structured(text2, schema, include_raw)

        # Runnable 로 래핑해서 반환 (체인 파이프에 바로 사용 가능)
        return RunnableLambda(_invoke)

In [ ]:
chat_model = GemmaChatModel(model=model, tokenizer=tokenizer, max_tokens=1024*5)

# 기본적인 구성 및 기능 - Runnable: invoke, batch, stream

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

In [ ]:
output_parser = StrOutputParser()

In [ ]:
chain = prompt | chat_model | output_parser

In [ ]:
output = chain.invoke({"dish": "카레"})
print(output)

In [ ]:
output = chain.batch([{"dish": "카레"}, {"dish": "우동"}])

In [ ]:
print(output[0])

In [ ]:
print(output[1])

In [ ]:
for chunk in chain.stream({"dish": "카레"}):
    print(chunk, end="", flush=True)

# Few Shot Like 구성해 보기

In [ ]:
gemma_model = GemmaChatModel(model=model, tokenizer=tokenizer, max_tokens=1024*5, verbose=True)

In [ ]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문에 단계적으로 답변하세요."),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | gemma_model | output_parser

In [ ]:
output = cot_chain.invoke({"question": "10 + 2 * 3"})
print(output)

In [ ]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "단계적으로 생각한 답변에서 결론만 추출하세요."),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | gemma_model | output_parser

In [ ]:
cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print(output)

# 함수를 Chain에 붙이기

In [ ]:
from langchain_core.runnables import RunnableLambda, chain

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

In [ ]:
def upper(text: str) -> str:
    return text.upper()

In [ ]:
chain_lambda = prompt | chat_model | output_parser | RunnableLambda(upper)

In [ ]:
ai_message = chain_lambda.invoke({"input": "Hello!"})
print(ai_message)

In [ ]:
@chain
def upper_deco(text: str) -> str:
    return text.upper()

In [ ]:
chain_deco = prompt | chat_model | output_parser | upper_deco

In [ ]:
ai_message = chain_deco.invoke({"input": "Hello!"})
print(ai_message)

# 함수를 Stream Chain에 붙이기

In [ ]:
def upper(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()

In [ ]:
chain_stream = prompt | chat_model | StrOutputParser() | upper

In [ ]:
for chunk in chain_stream.stream({"input": "Hello!"}):
    print(chunk, end="", flush=True)

# Parallel

In [ ]:
from langchain_core.runnables import RunnableParallel

## optimistic chain

In [ ]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 낙관주의자입니다. 사용자의 입력에 대해 낙관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)

In [ ]:
optimistic_chain = optimistic_prompt | chat_model | output_parser

## pessimistic chain

In [ ]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 비관주의자입니다. 사용자의 입력에 대해 비관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)

In [ ]:
pessimistic_chain = pessimistic_prompt | chat_model | output_parser

In [ ]:
parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

## 여럿 의견 수집

In [ ]:
output = parallel_chain.invoke({"topic": "생성 AI의 진화에 관해"})

In [ ]:
print(output["optimistic_opinion"])

In [ ]:
print(output["pessimistic_opinion"])

## 의견 종합

In [ ]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 객관적 AI입니다. 두 가지 의견을 종합하세요."),
        ("human", "낙관적 의견: {optimistic_opinion}\n비관적 의견: {pessimistic_opinion}"),
    ]
)

In [ ]:
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | chat_model
    | output_parser
)

In [ ]:
output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

In [ ]:
# RunnableParallel 자동 변환
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | chat_model
    | output_parser
)

In [ ]:
output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

# Parallel 사용 구조

In [ ]:
# operator는 파이썬의 연산자들을 “함수”로 제공하는 표준 라이브러리 모듈
from operator import itemgetter
import textwrap

In [ ]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 낙관주의자입니다. 사용자의 입력에 대해 낙관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)

In [ ]:
optimistic_chain = optimistic_prompt | chat_model | output_parser

In [ ]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 비관주의자입니다. 사용자의 입력에 대해 비관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)

In [ ]:
pessimistic_chain = pessimistic_prompt | chat_model | output_parser

In [ ]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            textwrap.dedent("""
                당신은 객관적 AI입니다. '{topic}'에 대한 두 가지 의견을 종합하세요.

                # 다음과 같은 일반적인 의견도 있습니다. 참고해 주세요.
                {general}
            """).strip()
        ),
        (
            "human",
            "낙관적 의견: {optimistic_opinion}\n비관적 의견: {pessimistic_opinion}",
        ),
    ]
)

In [ ]:
# invoke 에서 넘어오는 파라미터를 바로 사용해 보기
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic"),
        "general": itemgetter("general"),
    }
    | synthesize_prompt
    | chat_model
    | output_parser
)

In [ ]:
general_opinion = """
생성형 AI는 단순히 인간의 작업을 자동화하는 도구를 넘어, 창의성과 지식 생산의 범위를 폭발적으로 확장하는 기술입니다.
누구나 손쉽게 글·이미지·코드 등을 만들어낼 수 있게 함으로써 교육, 예술, 연구, 산업 전반에 혁신을 촉진합니다.
그러나 이와 동시에, 진위 판단의 어려움, 저작권과 데이터 소유권 문제, 일자리 구조 변화 등 사회 전반의 새로운 도전과 책임을 수반합니다.

따라서 생성형 AI의 사회적 의미는 단순한 기술 진보가 아니라, “인간과 기계가 어떻게 협력하고, 무엇을 신뢰하며, 어떤 가치를 지켜야 하는가”라는 집단적 선택의 시험대에 있다는 점입니다.
기술의 잠재력을 최대화하면서도 부작용을 최소화하려면, 투명성과 공정성, 그리고 지속적인 윤리 논의가 필수적입니다.
"""

In [ ]:
output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해", "general": general_opinion})
print(output)

# Tavily 검색

In [ ]:
!pip install tavily-python

In [ ]:
from tavily import TavilyClient
import json
import os

In [ ]:
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_KEY")

In [ ]:
client = TavilyClient()  # TAVILY_API_KEY를 env에서 읽음

In [ ]:
res = client.search("서울 오늘 날씨 관련 최신 기사 요약", max_results=3)
print(json.dumps(res, indent=2, ensure_ascii=False))

# Passthrough

In [ ]:
prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.runnables import RunnablePassthrough

In [ ]:
retriever = TavilySearchAPIRetriever(k=3)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser()
)

In [ ]:
output = chain.invoke("서울의 현재 날씨는? 비는 오나요?")
print(output)

## assign, pick

In [ ]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | chat_model | StrOutputParser())
    .pick(["context", "answer", "question"])
)

In [ ]:
output = chain.invoke("서울의 현재 날씨는?")
print(output)

In [ ]:
print(output["answer"])

# Database Memory

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import SQLChatMessageHistory
from uuid import uuid4
import sqlite3

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과거대화 내용을 바탕으로 상담을 해 주는 친절한 상담사 입니다."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

In [ ]:
chain = prompt | chat_model | StrOutputParser()

In [ ]:
def respond(session_id: str, human_message: str) -> str:
    chat_message_history = SQLChatMessageHistory(
        connection="sqlite:///sqlite.db",
        table_name="chat_store", # default: message_store
        session_id=session_id,
    )

    ai_message = chain.invoke(
        {
            "chat_history": chat_message_history.get_messages(),
            "input": human_message,
        }
    )

    chat_message_history.add_user_message(human_message)
    chat_message_history.add_ai_message(ai_message)

    return ai_message

In [ ]:
session_id = uuid4().hex
session_id

In [ ]:
output1 = respond(
    session_id=session_id,
    human_message="안녕하세요! 제 이름은 존이라고 합니다!",
)
print(output1)

In [ ]:
output2 = respond(
    session_id=session_id,
    human_message="제 이름을 알고 계신가요?",
)
print(output2)

In [ ]:
chat_message_history = SQLChatMessageHistory(
    connection="sqlite:///sqlite.db",
    table_name="chat_store", # default: message_store
    session_id=session_id,
)

In [ ]:
msgs = chat_message_history.get_messages()
for m in msgs:
    print(type(m), m.content)

In [ ]:
def get_all_session_ids(db_path="sqlite.db") -> List[str]:
    con = sqlite3.connect(db_path)
    rows = con.execute("SELECT DISTINCT session_id FROM chat_store").fetchall()
    con.close()
    return [r[0] for r in rows]

In [ ]:
for sid in get_all_session_ids():
    history = SQLChatMessageHistory(session_id=sid, connection="sqlite:///sqlite.db", table_name="chat_store")
    print(f"[+] session: {sid}")
    msgs = history.get_messages()
    for m in msgs:
        print(type(m), m.content)
    print("\n\n")